In [1]:
from bertviz import model_view, head_view
from transformers import AutoTokenizer, AutoModel
import pickle

# TODO : torch text should be 0.11.2 // torch = 1.10.2
from torchtext.datasets import IMDB
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
import torch





/home/kchen91/anaconda3/envs/former/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = get_tokenizer('basic_english')
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

In [3]:
train_iter = IMDB(split='train')
vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: 0 if x=='neg' else 1

In [5]:
sentence = "this is a book on the table"
processed_text = torch.tensor(text_pipeline(sentence), dtype=torch.int64)
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)


In [6]:
processed_text= processed_text.view(1, -1)

In [7]:
model.train(False)
out, att = model(processed_text)

In [10]:
att_reshaped = tuple([x[None,:] for x in att])

In [13]:
att_reshaped

(tensor([[[[0.1374, 0.1037, 0.1567, 0.1709, 0.1521, 0.1447, 0.1344],
           [0.1026, 0.2135, 0.1108, 0.1281, 0.1282, 0.1496, 0.1671],
           [0.1464, 0.1483, 0.1360, 0.1078, 0.2029, 0.1282, 0.1305],
           [0.1358, 0.1266, 0.0803, 0.1533, 0.1886, 0.1633, 0.1521],
           [0.1495, 0.1514, 0.1700, 0.1161, 0.1443, 0.1501, 0.1186],
           [0.1438, 0.2294, 0.1448, 0.0740, 0.1830, 0.1272, 0.0978],
           [0.1395, 0.1998, 0.1054, 0.1106, 0.2174, 0.1163, 0.1110]],
 
          [[0.1374, 0.1122, 0.1473, 0.1532, 0.1108, 0.1903, 0.1488],
           [0.1370, 0.1288, 0.1912, 0.1178, 0.1085, 0.1697, 0.1470],
           [0.1292, 0.1192, 0.1805, 0.1511, 0.1256, 0.1351, 0.1593],
           [0.1333, 0.1439, 0.1376, 0.1528, 0.1716, 0.1517, 0.1092],
           [0.1432, 0.1423, 0.1035, 0.1048, 0.1989, 0.1327, 0.1747],
           [0.1582, 0.1497, 0.1226, 0.1448, 0.1669, 0.1517, 0.1060],
           [0.1273, 0.1389, 0.1360, 0.1784, 0.0981, 0.1614, 0.1600]],
 
          [[0.1648, 0.1565, 

In [14]:
model_view(att_reshaped, tokenizer(sentence))

<IPython.core.display.Javascript object>